In [5]:
####PREPROCESSING####
import pandas as pd 

data = pd.read_csv('songdata.csv.zip')

artists = set(data['artist'])
kanyetext = data.loc[data['artist'] == 'Kanye West','text']
#, 'Drake', 'Mc Hammer', 'Eminem', 'Migos', 'Flo-Rida'
draketext = data.loc[data['artist'] == 'Drake','text']
eminemtext = data.loc[data['artist'] == 'Eminem','text']
migostext = data.loc[data['artist'] == 'Migos', 'text']
floridatext = data.loc[data['artist'] == 'Flo-Rida', 'text']
mchammertext = data.loc[data['artist'] == 'Mc Hammer', 'text']

text = [kanyetext, draketext, eminemtext, migostext, floridatext, mchammertext]
text = pd.concat(text)
print(len(text))

385


In [6]:
text = ''.join(text) 
text = text.lower()
print(len(text))
print(text[:1000])

952008
why are you so paranoid?  
don't be so paranoid  
don't be so...  
  
baby, don't worry about it  
hey there, don't even think about it  
  
you worry bout the wrong things, the wrong things  
you worry bout the wrong things, the wrong things  
you worry bout the wrong things, the wrong things  
you worry bout the wrong things, the wrong things  
  
all of the time, you really wanna spend your whole life alone  
a little time out might do ya good, might do us good before you're done for
good  
because i could make it good, i could make it hood, i could make you come, i
could make you gooo  
i could make you hot, i could make you fly, make you touch the sky, hey, maybe
sooo  
all of the time, he be up in my, checking through my cell phone, baby no  
you wanna kill the vibe, on another night, here's another fight, oh, here we
go (oh, here we go)  
  
baby, don't worry 'bout it  
lady, we'll go out to the floor  
  
anyway, they don't know you like i do (they don't know)  
they'll 

In [7]:
#LSTM implementation taken from https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys


chars = sorted(list(set(text))) #get all unique characters in the text
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars)) #map character to index
indices_char = dict((i, c) for i, c in enumerate(chars)) #map index to character

print(char_indices)

print(indices_char)

total chars: 50
{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, '?': 21, '[': 22, ']': 23, 'a': 24, 'b': 25, 'c': 26, 'd': 27, 'e': 28, 'f': 29, 'g': 30, 'h': 31, 'i': 32, 'j': 33, 'k': 34, 'l': 35, 'm': 36, 'n': 37, 'o': 38, 'p': 39, 'q': 40, 'r': 41, 's': 42, 't': 43, 'u': 44, 'v': 45, 'w': 46, 'x': 47, 'y': 48, 'z': 49}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '5', 16: '6', 17: '7', 18: '8', 19: '9', 20: ':', 21: '?', 22: '[', 23: ']', 24: 'a', 25: 'b', 26: 'c', 27: 'd', 28: 'e', 29: 'f', 30: 'g', 31: 'h', 32: 'i', 33: 'j', 34: 'k', 35: 'l', 36: 'm', 37: 'n', 38: 'o', 39: 'p', 40: 'q', 41: 'r', 42: 's', 43: 't', 44: 'u', 45: 'v', 46: 'w', 47: 'x', 48: 'y', 49: 'z'}


In [16]:
text = [kanyetext, draketext, eminemtext, migostext, floridatext, mchammertext]

for i in kanyetext:
    print(i)
"""for i in range(len(text)):
    text[i] = text[i].lower()
    
print(text[0])"""

Why are you so paranoid?  
Don't be so paranoid  
Don't be so...  
  
Baby, don't worry about it  
Hey there, don't even think about it  
  
You worry bout the wrong things, the wrong things  
You worry bout the wrong things, the wrong things  
You worry bout the wrong things, the wrong things  
You worry bout the wrong things, the wrong things  
  
All of the time, you really wanna spend your whole life alone  
A little time out might do ya good, might do us good before you're done for
good  
Because I could make it good, I could make it hood, I could make you come, I
could make you gooo  
I could make you hot, I could make you fly, make you touch the sky, Hey, maybe
sooo  
All of the time, he be up in my, checking through my cell phone, baby no  
You wanna kill the vibe, on another night, here's another fight, Oh, here we
go (Oh, here we go)  
  
Baby, don't worry 'bout it  
Lady, we'll go out to the floor  
  
Anyway, they don't know you like I do (They don't know)  
They'll never k

'for i in range(len(text)):\n    text[i] = text[i].lower()\n    \nprint(text[0])'

In [6]:
print(len(text))

57650


In [8]:
####TRAINING MODEL####
# cut the text in semi-redundant sequences of maxlen characters, split by song

maxlen = 40
step = 3
sentences = []
next_chars = []

#breaks text up like this:     
'''["hello darkness my old friend i've come t", 
    "lo darkness my old friend i've come to t", 
    "darkness my old friend i've come to talk", 
    "kness my old friend i've come to talk wi", 
    "ss my old friend i've come to talk with ", 
    "my old friend i've come to talk with you", 
    "old friend i've come to talk with you ag"] --> sentences, and then puts the next char in next_chars'''

for i in range(len(text)): 
    for j in range(0, len(text[i]) - maxlen, step):
        sentences.append(text[i][j: j + maxlen])
        next_chars.append(text[i][j + maxlen])
print('number of sequences:', len(sentences))

number of sequences: 21916700


In [9]:
print('Vectorization...')

#initialize two arrays, X = (maxlen rows x len(chars) cols) x len(sentences), y = len(sentences x len(chars))
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

#for every sequence, mark the appearance of every char in that sequence and then mark the corresponding next char for that sequence
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [10]:

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars))) 
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
  328960/21916700 [..............................] - ETA: 63346s - loss: 1.7564 

KeyboardInterrupt: 